In [1]:
# -*- coding: utf-8 -*-
"""SmolVLM_Benchmarks.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1xrpj5QUTActxwIqh3SVfJwUPp6VLwTt7
"""

# !pip install -q bitsandbytes accelerate quanto optimum-quanto

# from huggingface_hub import notebook_login

# notebook_login()

from transformers import AutoProcessor, QuantoConfig, AutoModelForVision2Seq, AutoModelForCausalLM, AutoTokenizer, AutoModel
import time
import gc
import requests
from PIL import Image
import time
import torch
import requests

device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
import numpy as np
import torch
import torchvision.transforms as T
from PIL import Image
from torchvision.transforms.functional import InterpolationMode
from transformers import AutoModel, AutoTokenizer

IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)

class InternVL2Processor:
    def __init__(self):
        self.tokenizer = AutoTokenizer.from_pretrained("OpenGVLab/InternVL2-2B", trust_remote_code=True, use_fast=False)

    def build_transform(self, input_size):
        MEAN, STD = IMAGENET_MEAN, IMAGENET_STD
        transform = T.Compose([
            T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
            T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
            T.ToTensor(),
            T.Normalize(mean=MEAN, std=STD)
        ])
        return transform

    def find_closest_aspect_ratio(self, aspect_ratio, target_ratios, width, height, image_size):
        best_ratio_diff = float('inf')
        best_ratio = (1, 1)
        area = width * height
        for ratio in target_ratios:
            target_aspect_ratio = ratio[0] / ratio[1]
            ratio_diff = abs(aspect_ratio - target_aspect_ratio)
            if ratio_diff < best_ratio_diff:
                best_ratio_diff = ratio_diff
                best_ratio = ratio
            elif ratio_diff == best_ratio_diff:
                if area > 0.5 * image_size * image_size * ratio[0] * ratio[1]:
                    best_ratio = ratio
        return best_ratio

    def dynamic_preprocess(self, image, min_num=1, max_num=12, image_size=448, use_thumbnail=False):
        orig_width, orig_height = image.size
        aspect_ratio = orig_width / orig_height

        # calculate the existing image aspect ratio
        target_ratios = set(
            (i, j) for n in range(min_num, max_num + 1) for i in range(1, n + 1) for j in range(1, n + 1) if
            i * j <= max_num and i * j >= min_num)
        target_ratios = sorted(target_ratios, key=lambda x: x[0] * x[1])

        # find the closest aspect ratio to the target
        target_aspect_ratio = self.find_closest_aspect_ratio(
            aspect_ratio, target_ratios, orig_width, orig_height, image_size)

        # calculate the target width and height
        target_width = image_size * target_aspect_ratio[0]
        target_height = image_size * target_aspect_ratio[1]
        blocks = target_aspect_ratio[0] * target_aspect_ratio[1]

        # resize the image
        resized_img = image.resize((target_width, target_height))
        processed_images = []
        for i in range(blocks):
            box = (
                (i % (target_width // image_size)) * image_size,
                (i // (target_width // image_size)) * image_size,
                ((i % (target_width // image_size)) + 1) * image_size,
                ((i // (target_width // image_size)) + 1) * image_size
            )
            # split the image
            split_img = resized_img.crop(box)
            processed_images.append(split_img)
        assert len(processed_images) == blocks
        if use_thumbnail and len(processed_images) != 1:
            thumbnail_img = image.resize((image_size, image_size))
            processed_images.append(thumbnail_img)
        return processed_images

    def load_image(self, image_url, input_size=448, max_num=12):
        image = Image.open(requests.get(image_url, stream=True).raw).convert('RGB')
        transform = self.build_transform(input_size=input_size)
        images = self.dynamic_preprocess(image, image_size=input_size, use_thumbnail=True, max_num=max_num)
        pixel_values = [transform(image) for image in images]
        pixel_values = torch.stack(pixel_values)
        return pixel_values

In [3]:
def get_gpu_memory():
    """Returns GPU memory usage in GB"""
    if torch.cuda.is_available():
        return torch.cuda.memory_allocated() / 1024**3
    return 0

def load_model(model_id):
    """
    Loads the model and measures memory usage.
    Returns the model, processor, and base memory usage.
    """
    # Clear any existing memory
    torch.cuda.empty_cache()
    gc.collect()

    initial_memory = get_gpu_memory()
    print(f"Initial GPU memory usage before loading: {initial_memory:.2f} GB")
    
    if "moondream" in model_id.lower():
        # Special case for Moondream
        processor = AutoTokenizer.from_pretrained(model_id, revision="2024-08-26")
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            trust_remote_code=True,
            revision="2024-08-26",
            device_map="cuda:0",
            torch_dtype=torch.bfloat16
        )
    elif 'internvl2' in model_id.lower():
        model = AutoModel.from_pretrained(
            model_id,
            torch_dtype=torch.bfloat16,
            low_cpu_mem_usage=True,
            use_flash_attn=True,
            trust_remote_code=True).eval().cuda()
        processor = InternVL2Processor()
    elif 'minicpm' in model_id.lower():
        model = AutoModel.from_pretrained('openbmb/MiniCPM-V-2', trust_remote_code=True, torch_dtype=torch.bfloat16)
        model = model.to(device='cuda', dtype=torch.bfloat16)
        processor = AutoTokenizer.from_pretrained('openbmb/MiniCPM-V-2', trust_remote_code=True)
        model.eval()
    else:
        # Original code for other models
        processor = AutoProcessor.from_pretrained(model_id)
        model = AutoModelForVision2Seq.from_pretrained(
            model_id,
            _attn_implementation="flash_attention_2",
            torch_dtype=torch.bfloat16,
            device_map="cuda:0",
        )
    
    
    base_memory = get_gpu_memory()
    print(f"GPU memory usage after loading: {base_memory:.2f} GB")
    print(f"GPU memory usage from loading the model: {base_memory-initial_memory:.2f} GB")
    
    return model, processor, base_memory

def benchmark_vlm(model_id, batch_size):
    torch.cuda.empty_cache()
    gc.collect()

    model, processor, base_memory = load_model(model_id)

    image_url = "https://huggingface.co/spaces/merve/chameleon-7b/resolve/main/bee.jpg"
    image = [Image.open(requests.get(image_url, stream=True).raw).convert('RGB')]

    images = []
    prompts = []
    num_patches_list = []
    pixel_values = None

    for _ in range(batch_size):
        images.append(image)
        if 'paligemma' in model_id.lower():
            prompt = "This is a photo of a"
        elif 'moondream' in model_id.lower():
            prompt = "This is a photo of a"
        elif 'internvl2' in model_id.lower():
            pixel_value = processor.load_image(image_url, max_num=24).to(torch.bfloat16).cuda()
            num_patches_list.append(pixel_value.size(0))
            if pixel_values is None:
                pixel_values = pixel_value
            else:
                pixel_values = torch.cat((pixel_values, pixel_value), dim=0)
            prompt = "<image>\nThis is a photo of a"
        elif 'minicpm' in model_id.lower():
            prompt = [{'role': 'user', 'content': "nThis is a photo of a"}]
        else:
            resulting_messages = [
                {
                    "role": "user",
                    "content": [{"type": "image"}] + [{"type": "text", "text": "This is a photo of a"}]
                }
            ]
            prompt = processor.apply_chat_template(resulting_messages, add_generation_prompt=True)
        prompts.append(prompt)

    if 'moondream' in model_id.lower() or 'internvl2' in model_id.lower() or 'minicpm' in model_id.lower():
        inputs = []
        prefill_tokens_per_sequence = -1
    else:
        inputs = processor(text=prompts, images=images, return_tensors="pt",
                       padding=True).to(device).to(torch.bfloat16)
        prefill_tokens_per_sequence = inputs.input_ids.size(1) # this is the input seq len basically
    
    total_prefill_tokens = prefill_tokens_per_sequence * batch_size

    # Measure memory before warmup
    memory_before_prefill = get_gpu_memory()
    print(f"Memory before prefill: {memory_before_prefill:.2f} GB")

    with torch.no_grad(): # warmup
        for _ in range(10):
            if 'moondream' in model_id.lower():
                _ = model.batch_answer(images[0], prompt, processor)
            elif 'internvl2' in model_id.lower():
                _ = model.batch_chat(processor.tokenizer, pixel_values,
                        num_patches_list=num_patches_list,
                        questions=prompts,
                        generation_config=dict(max_new_tokens=10, do_sample=True))
            elif 'minicpm' in model_id.lower():
                res, context, _ = model.chat(
                        image=image[0],
                        msgs=prompt,
                        context=None,
                        tokenizer=processor,
                        sampling=True,
                        temperature=0.7
                    )
            else:
                _ = model(**inputs)

    start_time = time.time()
    with torch.no_grad():
        if 'moondream' in model_id.lower():
            _ = model.batch_answer(images[0], prompt, processor)
        elif 'internvl2' in model_id.lower():
            _ = model.batch_chat(processor.tokenizer, pixel_values,
                num_patches_list=num_patches_list,
                questions=prompts,
                generation_config=dict(max_new_tokens=1, do_sample=True))
        elif 'minicpm' in model_id.lower():
            res, context, _ = model.chat(
                    image=image[0],
                    msgs=prompt,
                    context=None,
                    tokenizer=processor,
                    sampling=True,
                    temperature=0.7
                )
        else:
            output = model(**inputs)
    end_time = time.time()

    # Measure memory after prefill
    memory_after_prefill = get_gpu_memory()
    print(f"Memory after prefill: {memory_after_prefill:.2f} GB")
    
    prefill_duration = end_time - start_time
    prefill_tps = total_prefill_tokens / prefill_duration
    print(f"Prefill tokens per second: {prefill_tps:.2f}")
    print(f"Prefill memory usage: {memory_after_prefill:.2f} GB")

    gen_token_count_per_sequence = 200
    total_gen_tokens = gen_token_count_per_sequence * batch_size
    if 'moondream' in model_id.lower() or 'internvl2' in model_id.lower() or 'minicpm' in model_id.lower():
        input_ids = inputs
    else:
        input_ids = inputs.input_ids

    # Measure memory before generation
    memory_before_gen = get_gpu_memory()
    print(f"Memory before generation: {memory_before_gen:.2f} GB")

    start_time = time.time()
    output = []
    for _ in range(gen_token_count_per_sequence):
        with torch.no_grad():
            if 'moondream' in model_id.lower():
                _ = model.batch_answer(images[0], prompt, processor)
                break
            elif 'internvl2' in model_id.lower():
                _ = model.batch_chat(processor.tokenizer, pixel_values,
                    num_patches_list=num_patches_list,
                    questions=prompts,
                    generation_config=dict(max_new_tokens=gen_token_count_per_sequence, do_sample=True))
                break
            elif 'minicpm' in model_id.lower():
                res, context, _ = model.chat(
                        image=image[0],
                        msgs=prompt,
                        context=None,
                        tokenizer=processor,
                        sampling=True,
                        temperature=0.7
                    )
                break
            else:
                output = model(input_ids)
                next_token = torch.argmax(output.logits[:, -1, :], dim=-1, keepdim=True)
                input_ids = torch.cat([input_ids, next_token], dim=-1)
    end_time = time.time()

    # Measure memory after generation
    memory_after_gen = get_gpu_memory()
    print(f"Memory after generation: {memory_after_gen:.2f} GB")
    
    gen_duration = end_time - start_time
    gen_tps = total_gen_tokens / gen_duration
    print(f"Generated tokens per second: {gen_tps:.2f}")
    print(f"Generation memory usage: {memory_after_gen:.2f} GB")
    
    # Clean up after benchmark
    del output, input_ids
    torch.cuda.empty_cache()
    gc.collect()
    
    return prefill_tps, gen_tps, base_memory, memory_after_prefill, memory_after_gen


In [4]:
import torch
import time
import threading

model_ids = ["HuggingFaceTB/SmolVLM-Instruct", "Qwen/Qwen2-VL-2B-Instruct", "HuggingFaceM4/Idefics3-8B-Llama3", "OpenGVLab/InternVL2-2B", "google/paligemma-3b-mix-448", "vikhyatk/moondream2",
             "openbmb/MiniCPM-V-2"]

model_ids = ["openbmb/MiniCPM-V-2"]
# For MiniCPM to work we need to change the code as described here: https://huggingface.co/openbmb/MiniCPM-V-2/discussions/23

print(f"Initial memory: {torch.cuda.max_memory_allocated() / (1024**3):.2f} GB")

stats = {}
for model_id in model_ids:
    for batch_size in [1]:#, 4]:#, 32, 64]:
        print(f"\nTesting model: {model_id} with batch size: {batch_size}")
        torch.cuda.empty_cache()
        gc.collect()
        torch.cuda.torch.cuda.reset_peak_memory_stats()
        print(f"Current memory: {torch.cuda.max_memory_allocated() / (1024**3):.2f} GB")
        print(f"Max Memory Reserved: {torch.cuda.max_memory_reserved() / (1024**3):.2f} GB")

        prefill_tps, gen_tps, base_memory, prefill_mem, gen_mem = benchmark_vlm(model_id, batch_size)
        stats[(model_id, batch_size)] = {
            "prefill_tps": prefill_tps, 
            "gen_tps": gen_tps,
            "base_memory_gb": base_memory,
            "prefill_memory_gb": prefill_mem,
            "gen_memory_gb": gen_mem,
            "max_memory_gb": torch.cuda.max_memory_allocated() / (1024**3),
            "max_memory_reserved_gb": torch.cuda.max_memory_reserved() / (1024**3)
        }
        print(f"Max Memory: {torch.cuda.max_memory_allocated() / (1024**3):.2f} GB")
        print(f"Max Memory Reserved: {torch.cuda.max_memory_reserved() / (1024**3):.2f} GB")

        torch.cuda.empty_cache()
        gc.collect()

# Print final results
print("\nFinal Results:")
for (model_id, batch_size), metrics in stats.items():
    print(f"\nModel {model_id} with batch size {batch_size}:")
    print(f"Base model memory usage: {base_memory:.2f} GB")
    print(f"Prefill TPS: {metrics['prefill_tps']:.2f}")
    print(f"Generation TPS: {metrics['gen_tps']:.2f}")
    print(f"Additional Prefill Memory: {metrics['prefill_memory_gb']:.2f} GB")
    print(f"Additional Generation Memory: {metrics['gen_memory_gb']:.2f} GB")
    print(f"Max Memory: {metrics['max_memory_gb']:.2f} GB")


Initial memory: 0.00 GB

Testing model: openbmb/MiniCPM-V-2 with batch size: 1
Current memory: 0.00 GB
Max Memory Reserved: 0.00 GB
Initial GPU memory usage before loading: 0.00 GB


MiniCPMForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

GPU memory usage after loading: 6.53 GB
GPU memory usage from loading the model: 6.53 GB
Memory before prefill: 6.53 GB


The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48


Memory after prefill: 6.56 GB
Prefill tokens per second: -0.48
Prefill memory usage: 6.56 GB
Memory before generation: 6.56 GB
Memory after generation: 6.56 GB
Generated tokens per second: 88.80
Generation memory usage: 6.56 GB
Max Memory: 7.36 GB
Max Memory Reserved: 7.88 GB

Final Results:

Model openbmb/MiniCPM-V-2 with batch size 1:
Base model memory usage: 6.53 GB
Prefill TPS: -0.48
Generation TPS: 88.80
Additional Prefill Memory: 6.56 GB
Additional Generation Memory: 6.56 GB
Max Memory: 7.36 GB


In [12]:
import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForVision2Seq
from transformers.image_utils import load_image

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Load images
image1 = load_image("https://cdn.britannica.com/61/93061-050-99147DCE/Statue-of-Liberty-Island-New-York-Bay.jpg")
image2 = load_image("https://huggingface.co/spaces/merve/chameleon-7b/resolve/main/bee.jpg")

# Initialize processor and model
processor = AutoProcessor.from_pretrained("HuggingFaceTB/SmolVLM-Instruct")
model = AutoModelForVision2Seq.from_pretrained(
    "HuggingFaceTB/SmolVLM-Instruct", torch_dtype=torch.bfloat16
).to(DEVICE)

# Create input messages
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": "What do we see in this image?"}
        ]
    },
    {
        "role": "assistant",
        "content": [
            {"type": "text", "text": "This image shows a city skyline with prominent landmarks."}
        ]
    },
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": "And how about this image?"}
        ]
    }
]

# Prepare inputs
prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(text=prompt, images=[image1, image2], return_tensors="pt")
inputs = {k: v.to(DEVICE) for k, v in inputs.items()}

# Generate outputs
generated_ids = model.generate(**inputs, max_new_tokens=500)
generated_texts = processor.batch_decode(generated_ids, skip_special_tokens=True)

print(generated_texts[0])


Some kwargs in processor config are unused and will not have any effect: image_seq_len. 


<|begin_of_text|>User:<image>What do we see in this image?
Assistant: This image shows a city skyline with prominent landmarks.
User:<image>And how about this image?
Assistant: In this image, there is a bee on a pink flower.
